In [28]:
# import shutil

# put 001.Plastic_bag in the PlasticDetection/images folder for more consistent
# shutil.copytree('unzipped/001.Plastic_bag/', 'PlasticDectection/images/001.Plastic_bag')
# shutil.copy('unzipped/test.lst', '/root')
# shutil.move('unzipped/train.lst', '/root')

'/root/train.lst'

## Install libs

In [ ]:
# platform.linux_distribution() is deprecated --> use distro package
!pip install distro

In [ ]:
import distro

if distro.linux_distribution(full_distribution_name=False)[0] == 'debian':
    ! apt-get update
    ! apt-get install ffmpeg libsm6 libxext6 -y

### opencv-python and mxnet

In [ ]:
import sys
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install mxnet

### Re-install numpy to run im2rec.py file

In [10]:
import numpy
numpy.__version__

'1.25.1'

In [ ]:
'''
!python tools/im2rec.py --resize $RESIZE_SIZE --pack-label test $BASE_DIR/images/
--> /opt/conda/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning:
A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy 
(detected version 1.25.2
'''
!pip install numpy==1.23.1

## Create RecordIO files  
After running, having train.rec and test.rec files

In [16]:
RESIZE_SIZE = 256
BASE_DIR = 'PlasticDetection/'

In [19]:
!python tools/im2rec.py --resize $RESIZE_SIZE --pack-label test $BASE_DIR/images/

# Eg. PlasticDetection/images/001.Plastic_bag/images/test/888fea91e5a0e984.jpg
# 001.Plastic_bag/images/test/888fea91e5a0e984.jpg is from test.lst

Creating .rec file from /root/test.lst in /root
multiprocessing not available, fall back to single threaded encoding
time: 0.027895212173461914  count: 0


In [20]:
!python tools/im2rec.py --resize $RESIZE_SIZE --pack-label train $BASE_DIR/images/

Creating .rec file from /root/train.lst in /root
multiprocessing not available, fall back to single threaded encoding
time: 0.06548452377319336  count: 0
time: 31.482555866241455  count: 1000


## Upload RecordIO files to S3 bucket  
Create a bucket on S3. Eg. bucket name: plastic-bag-object-detection-2023

In [21]:
import sagemaker

bucket = 'plastic-bag-object-detection-2023'
prefix = 'Object-Detection-Demo'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [22]:
from sagemaker import get_execution_role

role = get_execution_role()
role

'arn:aws:iam::261340088694:role/service-role/AmazonSageMaker-ExecutionRole-20230822T010758'

In [23]:
sess = sagemaker.Session()
sess

In [24]:
train_channel = prefix + '/train'
sess.upload_data(path='train.rec', bucket=bucket, key_prefix=train_channel)
s3_train_data = f's3://{bucket}/{train_channel}'
s3_train_data

's3://plastic-bag-object-detection-2023/Object-Detection-Demo/train'

In [25]:
validation_channel = prefix + '/validation'
sess.upload_data(path='test.rec', bucket=bucket, key_prefix=validation_channel)
s3_validation_data = f's3://{bucket}/{validation_channel}'
s3_validation_data

's3://plastic-bag-object-detection-2023/Object-Detection-Demo/validation'

## Download Object Dectection Algo from AWS ECR (Elastic Container Registry)

In [26]:
s3_output_location =  f's3://{bucket}/{prefix}/output'
s3_output_location

's3://plastic-bag-object-detection-2023/Object-Detection-Demo/output'

In [31]:
from sagemaker import image_uris

training_image = image_uris.retrieve(
    region=sess.boto_region_name,
    framework='object-detection',
    version='1' # '1' is latest on AWS
)
print(training_image)

811284229777.dkr.ecr.us-east-1.amazonaws.com/object-detection:1


## Setting up Estimator Object  
Estimators: Encapsulate training on SageMaker.

In [33]:
object_detection_model = sagemaker.estimator.Estimator(
    training_image=training_image,
    role=role,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    volume_size=50, # GB
    max_run=360000,  # seconds. 360000s=100 hours
    input_mode='File', # copy training data to EFS which is attached to Sagemaker notebook
                       # If using 'Pipe': can stream data directly from S3
    output_path = s3_output_location,
    sagemaker_session=sess,
)

print(object_detection_model)

TypeError: Estimator.__init__() missing 1 required positional argument: 'image_uri'